참고문헌 : Pytel 외 저, 이주성 외 역, 재료역학, 2판, 한티미디어, 2013.<br>Ref: Pytel, Kiusalaas, Sharma, Mechanics of Materials, 2nd Ed., Cengage Learning, 2013.

`python` 기능을 확장해 주는 `module`을 불러 들임 (일부 기능만 사용될 수도 있음)<br>
Bring in `module`'s that would expand features of `python`. (This file may use just some of them.)

In [ ]:
import numpy as np  # 배열, 행렬 관련 기능
import numpy.linalg as na  # 선형대수 (벡터, 행렬) 관련 기능
import matplotlib.pyplot as plt  # 그래프 관련 기능
import scipy.integrate as si  # 적분 관련 기능
import sympy as sy  # 기호 연산 기능
import IPython.display as disp  # 웹페이지 표시 기능
sy.init_printing()  # 기호 연산 결과 표시 기능 준비

## 예제 03.001

### 문제에서 주어진 매개변수

In [ ]:
L_AB_m, L_BC_m, d_m, T_A_Nm, T_B_Nm, G_Pa = sy.symbols('L_{AB}[m], L_{BC}[m], d[m], T_{A}[Nm], T_{B}[Nm], G[Pa]')

In [ ]:
d_mm = d_m * 1e3

In [ ]:
subs_dict = {L_AB_m: 2.5, # A-B 구간의 길이
             L_BC_m: 1.5, # B-C 구간의 길이
             d_mm: 60.0, # 실린더 지름
             T_A_Nm: 3600.0, # A 지점에서의 비틀림 하중
             T_B_Nm: -1600.0, # B 지점에서의 비틀림 하중
             G_Pa: 80e9, # 전단탄성계수
            }
subs_dict[d_m] = subs_dict[d_mm] * 1e-3 # 실린더 지름

### 1) 각 원통 A-B, B-C 에서의 최대 응력

#### C점에서의 반력

C 점에서의 반력을 `R_C` 라고 정한다.

In [ ]:
R_C_Nm = sy.symbols('R_{C}[Nm]')

평형을 유지하기 위해 A, B, C 지점에서의 비틀림 하중과 반력을 모두 합하면 0이 될 것이다.
$$T_A + T_B + R_C = 0$$
이것을 python 으로는 다음과 같이 표현할 수 있다.

In [ ]:
eq_eq = sy.Eq(T_A_Nm + T_B_Nm + R_C_Nm, 0)

이 식을 풀면 반력을 구할 수 있다. (아래에서 `_` 는 바로 위 결과를 말함)

In [ ]:
R_C_Sol = sy.solve(eq_eq, R_C_Nm)
disp.display(R_C_Sol)

#### 반지름과 단면의 극관성모멘트

반지름 r 은 다음과 같이 구할 수 있다.

In [ ]:
r_m = d_m * 0.5

원형 단면의 비틀림인 경우 극관성모멘트를 구한다.

In [ ]:
J_m4 = (sy.pi * d_m**4) / 32

#### 최대전단응력

A-B 구간에서의 최대전단응력은 다음과 같이 구할 수 있을 것이다.

In [ ]:
tau_max_AB_Pa = T_A_Nm * r_m / J_m4

B-C 구간에서의 최대전단응력도 마찬가지로 구할 수 있다.

In [ ]:
tau_max_BC_Pa = (T_A_Nm + T_B_Nm) * r_m / J_m4
disp.display(sy.simplify(tau_max_BC_Pa))

각 변수에 값을 대입하면 다음과 같다.

In [ ]:
tau_AB_Pa = float(tau_max_AB_Pa.subs(subs_dict))
disp.display(disp.Math('$\\tau_{maxAB}=%g(Pa)$' % tau_AB_Pa))
tau_AB_MPa = 1e-6*tau_AB_Pa
disp.display(disp.Math('$\\tau_{maxAB}=%g(MPa)$'% tau_AB_MPa))

In [ ]:
tau_BC_Pa = float(tau_max_BC_Pa.subs(subs_dict))
disp.display(disp.Math('$\\tau_{maxBC}=%g(Pa)$' % tau_BC_Pa))
disp.display(disp.Math('$\\tau_{maxBC}=%g(MPa)$'%(1e-6 * tau_BC_Pa)))

### 2) 끝점 A 에서의 변형량

축방향 하중의 경우 변형량은 아래와 같았다.
$$\delta=\frac{PL}{EA}$$

비틀림 하중의 경우 변형량은 다음과 같다.
$$\theta = \frac{TL}{GJ}$$

A-B 구간, B-C 구간의 변형량은 각각 다음과 같다.

In [ ]:
theta_AB_rad = T_A_Nm * L_AB_m / (G_Pa * J_m4)
disp.display(theta_AB_rad)

In [ ]:
theta_BC_rad = (T_A_Nm + T_B_Nm) * L_BC_m / (G_Pa * J_m4)
disp.display(theta_BC_rad)

각각 변수 값을 대입 하면 다음과 같다.

In [ ]:
disp.display(disp.Math('$\\theta_{AB}=%g(rad)$'%float(theta_AB_rad.subs(subs_dict))))
disp.display(disp.Math('$\\theta_{AB}=%g(deg)$'%(float((180/sy.pi)*theta_AB_rad.subs(subs_dict)))))

In [ ]:
theta_B_rad = float(theta_BC_rad.subs(subs_dict))
theta_B_deg = np.rad2deg(theta_B_rad)
disp.display(disp.Math('$\\theta_{BC}=%g(rad)$'% theta_B_rad))
disp.display(disp.Math('$\\theta_{BC}=%g(deg)$'% theta_B_deg))

전체 변형량은 다음과 같다.

In [ ]:
theta_AC_rad = sy.simplify(theta_AB_rad + theta_BC_rad)
disp.display(theta_AC_rad)

In [ ]:
theta_A_rad = float(theta_AC_rad.subs(subs_dict))
disp.display(disp.Math('$\\theta_{AC}=%g(rad)$'% theta_A_rad))
theta_A_deg = theta_A_rad * (180/sy.pi)
disp.display(disp.Math('$\\theta_{AC}=%g(deg)$'% theta_A_deg))

### 1) 각 원통 A-B, B-C 에서의 최대 응력 : 그림으로 표시해 보자

#### 비틀림 하중 선도

C 점의 x 좌표를 0으로 하자. C-B, B-A 각 구간을 다음과 같이 준비할 수 있다.

In [ ]:
x_CB_m_array = np.linspace(0, subs_dict[L_BC_m], 101)
x_BA_m_array = np.linspace(0, subs_dict[L_AB_m], 101) + subs_dict[L_BC_m]

각 구간 안의 x 좌표의 수는 아래와 같이 알 수 있다.

In [ ]:
(x_CB_m_array.shape, x_BA_m_array.shape)

두 구간을 다음과 같이 하나로 합칠 수도 있다.

In [ ]:
x_m_array = np.hstack((x_CB_m_array, x_BA_m_array))
x_m_array.shape

B-A 구간에서는 $T_{A}(Nm)$ 이 작용하고 있다. 위에서 정한 B-A 구간의 x 좌표 수와 같은 수의 1 을 담은 배열 `array` 을 만든 후 $T_A(Nm)$ 을 곱한다.

In [ ]:
T_BA_Nm_array = np.ones_like(x_BA_m_array) * subs_dict[T_A_Nm]

C-B 구간에서는 $T_{B}(Nm)$ 이 작용하고 있으므로 이번에도 위에서 정한 C-B 구간의 x 좌표 수와 같은 수의 1 을 담은 배열 `array` 을 만든 후 $T_A(Nm)+T_B(Nm)$ 을 곱한다.

In [ ]:
T_CB_Nm_array = np.ones_like(x_CB_m_array) * (subs_dict[T_A_Nm] + subs_dict[T_B_Nm])

비틀림 하중 `array`도 하나로 합쳐 준다.

In [ ]:
T_Nm_array = np.hstack((T_CB_Nm_array, T_BA_Nm_array))

이렇게 준비한 배열을 그래프로 표시하되, 선도 아래 부분이 칠해지도록 `fill_between()` 기능을 이용해본다.

In [ ]:
plt.clf()  # 이전에 그려진 그림이 있었다면 지우고 새로 시작한다.
plt.fill_between(x_CB_m_array, T_CB_Nm_array)
plt.fill_between(x_BA_m_array, T_BA_Nm_array)

A, B, C 점을 표시한다.

In [ ]:
plt.text(x_CB_m_array[0], 0, 'C')
plt.text(x_BA_m_array[0], 0, 'B')
plt.text(x_BA_m_array[-1], 0, 'A')

x y 축 제목을 붙인다.

In [ ]:
plt.xlabel('x(m)')
plt.ylabel('T(Nm)')

In [ ]:
plt.grid(True)
plt.show()

#### 비틀림 변위 선도

* 변형량은 변형률의 적분
* 변형률은 응력을 탄성계수로 나눈 것
* 비틀림 응력은 비틀림 하중에 반지름을 곱하고 단면계수 극관성모멘트로 나눈 것

##### 응력 선도

응력을 구하기 위해 비틀림 하중 $T$ 에 반지름 $r$을 곱하고 극관성모멘트 $J$로 나눈다.

In [ ]:
Pa2MPa = 1e-6

stress_Pa_array = T_Nm_array * float((r_m / J_m4).subs(subs_dict))
plt.clf()
plt.plot(x_m_array, stress_Pa_array * Pa2MPa)

위에서 구했던 A-B, B-C 구간의 응력을 투명도 (`alpha`) 0.5 로 표시한다.

In [ ]:
plt.plot([x_m_array[0], x_m_array[-1]], [tau_AB_Pa * Pa2MPa, tau_AB_Pa * Pa2MPa], alpha=0.5)
plt.plot([x_m_array[0], x_m_array[-1]], [tau_BC_Pa * Pa2MPa, tau_BC_Pa * Pa2MPa], alpha=0.5)

`A`, `B`, `C` 지점을 표시한다.

In [ ]:
plt.text(x_CB_m_array[0], tau_BC_Pa * Pa2MPa, 'C')
plt.text(x_BA_m_array[0], tau_BC_Pa * Pa2MPa, 'B')
plt.text(x_BA_m_array[-1], tau_AB_Pa * Pa2MPa, 'A')

`y` 축 최소값을 0으로 맞춘다.

In [ ]:
plt.ylim(ymin=0)

선도를 표시한다.

In [ ]:
plt.xlabel('x(m)')
plt.ylabel('$\\tau_{max}(MPa)$')
plt.grid(True)
plt.show()

### 2) 끝점 A 에서의 변형량

변형량의 $x$ 에 대한 미분을 계산한다.
\begin{equation}
\tau=G\gamma = G \frac{d\theta}{dx}\rho\\
\frac{d\theta}{dx} = \frac{\tau}{G \rho}
\end{equation}

In [ ]:
dtheta_dx_array = (stress_Pa_array / (G_Pa * r_m).subs(subs_dict))
plt.clf()  # 이전에 그려진 그림이 있었다면 지우고 새로 시작한다.
plt.plot(x_m_array, dtheta_dx_array)

# A, B, C 지점을 표시한다
plt.text(x_CB_m_array[0], tau_BC_Pa/(G_Pa * r_m).subs(subs_dict), 'C')
plt.text(x_BA_m_array[0], tau_BC_Pa/(G_Pa * r_m).subs(subs_dict), 'B')
plt.text(x_BA_m_array[-1], tau_AB_Pa/(G_Pa * r_m).subs(subs_dict), 'A')

plt.ylim(ymin=0)
plt.xlabel('x(m)')
plt.ylabel('$\\frac{d\\theta}{dx}(rad/m)$')
plt.grid(True)
plt.show()

변형률을 적분하여 변형량을 계산한다.

In [ ]:
displacement_rad_array = si.cumtrapz(dtheta_dx_array, x_m_array, initial=0)
displacement_deg_array = displacement_rad_array * (180/np.pi)

plt.clf()  # 이전에 그려진 그림이 있었다면 지우고 새로 시작한다.
plt.plot(x_m_array, displacement_deg_array, label='%\\theta(deg)%')
plt.plot([x_m_array[0], x_m_array[-1]], [theta_B_deg, theta_B_deg], alpha = 0.5)
plt.plot([x_m_array[0], x_m_array[-1]], [theta_A_deg, theta_A_deg], alpha = 0.5)

# A, B, C 지점을 표시한다
plt.text(x_CB_m_array[0], 0, 'C')
plt.text(x_BA_m_array[0], theta_B_deg, 'B')
plt.text(x_BA_m_array[-1], theta_A_deg, 'A')

plt.xlabel('x(m)')
plt.ylabel('$\\theta(deg)$')
plt.grid(True)
plt.show()

print(float(theta_A_deg / displacement_deg_array[-1]))

아래는 프로그램이 맞게 작동했는지 확인하기 위해 추가된 부분이다. 

참고 : Francesco Mosconi, Travis + Anaconda + Jupyter, 2017 Aug 09, [Online] Available: https://github.com/ghego/travis_anaconda_jupyter.

In [ ]:
assert(1e-7 > abs(theta_A_deg - displacement_deg_array[-1]))